In [1]:
# Modules to support development
import os
import re
import collections
import itertools
import functools
import logging
import pprint
import numpy as np
import heapq
import re
import nographs as nog
import copy

In [2]:
def read_input(puzzle_input):
    with open(puzzle_input) as ff:
        dd = [ ll.strip() for ll in ff.readlines() ]

    height = len(dd)
    width = len(dd[0])

    map = np.chararray((height, width))
    blizzards = {}
    start = None
    end = None

    for yy, ll in enumerate(dd):
        for xx, cc in enumerate(ll):            
            map[yy,xx] = cc
            if yy == 0 and cc == '.':
                start = yy, xx
            elif yy == height-1 and cc == '.':
                end = yy, xx
            elif cc in ('>', '<', 'v', '^'):
                blizzards.setdefault((yy,xx), []).append(cc)

    return map, blizzards, start, end

map, blizzards, start, end = read_input(os.path.join(os.path.join("..", "dat", "day24_test.txt")))
print(map)
print(blizzards)
print(start)
print(end)

[[b'#' b'.' b'#' b'#' b'#' b'#' b'#' b'#']
 [b'#' b'>' b'>' b'.' b'<' b'^' b'<' b'#']
 [b'#' b'.' b'<' b'.' b'.' b'<' b'<' b'#']
 [b'#' b'>' b'v' b'.' b'>' b'<' b'>' b'#']
 [b'#' b'<' b'^' b'v' b'^' b'^' b'>' b'#']
 [b'#' b'#' b'#' b'#' b'#' b'#' b'.' b'#']]
{(1, 1): ['>'], (1, 2): ['>'], (1, 4): ['<'], (1, 5): ['^'], (1, 6): ['<'], (2, 2): ['<'], (2, 5): ['<'], (2, 6): ['<'], (3, 1): ['>'], (3, 2): ['v'], (3, 4): ['>'], (3, 5): ['<'], (3, 6): ['>'], (4, 1): ['<'], (4, 2): ['^'], (4, 3): ['v'], (4, 4): ['^'], (4, 5): ['^'], (4, 6): ['>']}
(0, 1)
(5, 6)


In [16]:
def next_blizzards(shape, blizzards):
    _tmp = {}
    for (yy, xx), bbs in blizzards.items():
        for bb in bbs:
            ny, nx = yy, xx
            if bb == '>':
                nx = xx + 1
                if nx == shape[1] - 1:
                    nx = 1
            elif bb == '<':
                nx = xx - 1
                if nx == 0:
                    nx = shape[1] - 2
            elif bb == '^':
                ny = yy - 1
                if ny == 0:
                    ny = shape[0] - 2
            elif bb == 'v':
                ny = yy + 1
                if ny == shape[0] - 1:
                    ny = 1
            _tmp.setdefault((ny,nx),[]).append(bb)

    return _tmp

def print_blizzards(shape,blizzards):
    for yy in range(shape[0]):
        for xx in range(shape[1]):
            if yy in (0, shape[0]-1) or xx in (0, shape[1]-1):
                print("#", end="")
            elif (yy, xx) in blizzards:
                bbs = blizzards.get((yy,xx))
                if len(bbs) > 1:
                    print(len(bbs), end="")
                else:
                    print(bbs[0], end="")
            else:
                print('.', end="")
            
        print()

def part1(puzzle_input):
    map, blizzards, start, end = read_input(puzzle_input)

    print(start,end)
    map_cache = {}
    map_cache[0] = blizzards
    map_cache[1] = next_blizzards(map.shape, map_cache[0])
    map_cache[2] = next_blizzards(map.shape, map_cache[1])
    #print_blizzards(map.shape, map_cache[0])
    #print_blizzards(map.shape, map_cache[1])
    #print_blizzards(map.shape, map_cache[2])

    def next_edges(state, t):
        for ii in range(t.depth+2):
            if ii not in map_cache:
                print("Generating map at depth", ii)
                map_cache[ii] = next_blizzards(map.shape, map_cache[ii-1])
                #print_blizzards(map.shape, map_cache[ii])

        blizzards = map_cache[t.depth+1]

        yy, xx, _ = state
        if (yy,xx) == end:
            return
        for (dy, dx) in ((-1,0),(1,0),(0,-1),(0,1),(0,0)):
            ny, nx = (yy+dy, xx+dx)
            if (ny,nx) == end:
                # reached end
                print("Reached Goal!")
                yield (ny, nx, t.depth+1), 1
            elif (ny,nx) == start:
                yield (ny, nx, t.depth+1), 1
            elif ny < 1 or ny > map.shape[0] - 2:
                continue
            elif nx < 1 or nx > map.shape[1] - 2:
                continue
            elif (ny, nx) in blizzards:
                continue

            yield (ny, nx, t.depth+1), 1

    traversal = nog.TraversalShortestPaths(next_edges)
    for vertex in traversal.start_from((start[0], start[1], 0), build_paths=True):
        if vertex[0:2] == end:
            #print(len(traversal.paths[vertex]) #, traversal.paths[vertex])
            break
    #print(traversal.distance)
    #print(traversal.paths[found])

    return traversal.distance

#ans = part1(os.path.join(os.path.join("..", "dat", "day24_test.txt")))
#assert ans == 18

ans = part1(os.path.join(os.path.join("..", "dat", "day24.txt")))
print(ans)

(0, 1) (36, 100)
Generating map at depth 3
Generating map at depth 4
Generating map at depth 5
Generating map at depth 6
Generating map at depth 7
Generating map at depth 8
Generating map at depth 9
Generating map at depth 10
Generating map at depth 11
Generating map at depth 12
Generating map at depth 13
Generating map at depth 14
Generating map at depth 15
Generating map at depth 16
Generating map at depth 17
Generating map at depth 18
Generating map at depth 19
Generating map at depth 20
Generating map at depth 21
Generating map at depth 22
Generating map at depth 23
Generating map at depth 24
Generating map at depth 25
Generating map at depth 26
Generating map at depth 27
Generating map at depth 28
Generating map at depth 29
Generating map at depth 30
Generating map at depth 31
Generating map at depth 32
Generating map at depth 33
Generating map at depth 34
Generating map at depth 35
Generating map at depth 36
Generating map at depth 37
Generating map at depth 38
Generating map at 

In [19]:
def next_blizzards(shape, blizzards):
    _tmp = {}
    for (yy, xx), bbs in blizzards.items():
        for bb in bbs:
            ny, nx = yy, xx
            if bb == '>':
                nx = xx + 1
                if nx == shape[1] - 1:
                    nx = 1
            elif bb == '<':
                nx = xx - 1
                if nx == 0:
                    nx = shape[1] - 2
            elif bb == '^':
                ny = yy - 1
                if ny == 0:
                    ny = shape[0] - 2
            elif bb == 'v':
                ny = yy + 1
                if ny == shape[0] - 1:
                    ny = 1
            _tmp.setdefault((ny,nx),[]).append(bb)

    return _tmp

def print_blizzards(shape,blizzards):
    for yy in range(shape[0]):
        for xx in range(shape[1]):
            if yy in (0, shape[0]-1) or xx in (0, shape[1]-1):
                print("#", end="")
            elif (yy, xx) in blizzards:
                bbs = blizzards.get((yy,xx))
                if len(bbs) > 1:
                    print(len(bbs), end="")
                else:
                    print(bbs[0], end="")
            else:
                print('.', end="")
            
        print()

def find_path(shape, blizzards, start, end):
    print(start,end)
    map_cache = {}
    map_cache[0] = blizzards
    map_cache[1] = next_blizzards(shape, map_cache[0])
    map_cache[2] = next_blizzards(shape, map_cache[1])
    #print_blizzards(map.shape, map_cache[0])
    #print_blizzards(map.shape, map_cache[1])
    #print_blizzards(map.shape, map_cache[2])

    def next_edges(state, t):
        for ii in range(t.depth+2):
            if ii not in map_cache:
                print("Generating map at depth", ii)
                map_cache[ii] = next_blizzards(shape, map_cache[ii-1])
                #print_blizzards(map.shape, map_cache[ii])

        blizzards = map_cache[t.depth+1]

        yy, xx, _ = state
        if (yy,xx) == end:
            return
        for (dy, dx) in ((-1,0),(1,0),(0,-1),(0,1),(0,0)):
            ny, nx = (yy+dy, xx+dx)
            if (ny,nx) == end:
                # reached end
                print("Reached Goal!")
                yield (ny, nx, t.depth+1), 1
            elif (ny,nx) == start:
                yield (ny, nx, t.depth+1), 1
            elif ny < 1 or ny > shape[0] - 2:
                continue
            elif nx < 1 or nx > shape[1] - 2:
                continue
            elif (ny, nx) in blizzards:
                continue

            yield (ny, nx, t.depth+1), 1

    traversal = nog.TraversalShortestPaths(next_edges)
    for vertex in traversal.start_from((start[0], start[1], 0), build_paths=True):
        if vertex[0:2] == end:
            #print(len(traversal.paths[vertex]) #, traversal.paths[vertex])
            break
    #print(traversal.distance)
    #print(traversal.paths[found])

    return traversal.distance

def part1(puzzle_input):
    map, blizzards, start, end = read_input(puzzle_input)
    return find_path(map.shape, blizzards, start, end)
#ans = part1(os.path.join(os.path.join("..", "dat", "day24_test.txt")))
#assert ans == 18

ans = part1(os.path.join(os.path.join("..", "dat", "day24.txt")))
print(ans)

(0, 1) (36, 100)
Generating map at depth 3
Generating map at depth 4
Generating map at depth 5
Generating map at depth 6
Generating map at depth 7
Generating map at depth 8
Generating map at depth 9
Generating map at depth 10
Generating map at depth 11
Generating map at depth 12
Generating map at depth 13
Generating map at depth 14
Generating map at depth 15
Generating map at depth 16
Generating map at depth 17
Generating map at depth 18
Generating map at depth 19
Generating map at depth 20
Generating map at depth 21
Generating map at depth 22
Generating map at depth 23
Generating map at depth 24
Generating map at depth 25
Generating map at depth 26
Generating map at depth 27
Generating map at depth 28
Generating map at depth 29
Generating map at depth 30
Generating map at depth 31
Generating map at depth 32
Generating map at depth 33
Generating map at depth 34
Generating map at depth 35
Generating map at depth 36
Generating map at depth 37
Generating map at depth 38
Generating map at 

In [23]:
def next_blizzards(shape, blizzards):
    _tmp = {}
    for (yy, xx), bbs in blizzards.items():
        for bb in bbs:
            ny, nx = yy, xx
            if bb == '>':
                nx = xx + 1
                if nx == shape[1] - 1:
                    nx = 1
            elif bb == '<':
                nx = xx - 1
                if nx == 0:
                    nx = shape[1] - 2
            elif bb == '^':
                ny = yy - 1
                if ny == 0:
                    ny = shape[0] - 2
            elif bb == 'v':
                ny = yy + 1
                if ny == shape[0] - 1:
                    ny = 1
            _tmp.setdefault((ny,nx),[]).append(bb)

    return _tmp

def print_blizzards(shape,blizzards):
    for yy in range(shape[0]):
        for xx in range(shape[1]):
            if yy in (0, shape[0]-1) or xx in (0, shape[1]-1):
                print("#", end="")
            elif (yy, xx) in blizzards:
                bbs = blizzards.get((yy,xx))
                if len(bbs) > 1:
                    print(len(bbs), end="")
                else:
                    print(bbs[0], end="")
            else:
                print('.', end="")
            
        print()

def find_path(shape, blizzards, start, end):
    print(start,end)
    map_cache = {}
    map_cache[0] = blizzards
    map_cache[1] = next_blizzards(shape, map_cache[0])
    map_cache[2] = next_blizzards(shape, map_cache[1])
    #print_blizzards(map.shape, map_cache[0])
    #print_blizzards(map.shape, map_cache[1])
    #print_blizzards(map.shape, map_cache[2])

    def next_edges(state, t):
        for ii in range(t.depth+2):
            if ii not in map_cache:
                print("Generating map at depth", ii)
                map_cache[ii] = next_blizzards(shape, map_cache[ii-1])
                #print_blizzards(map.shape, map_cache[ii])

        blizzards = map_cache[t.depth+1]

        yy, xx, _ = state
        if (yy,xx) == end:
            return
        for (dy, dx) in ((-1,0),(1,0),(0,-1),(0,1),(0,0)):
            ny, nx = (yy+dy, xx+dx)
            if (ny,nx) == end:
                # reached end
                print("Reached Goal!")
                yield (ny, nx, t.depth+1), 1
            elif (ny,nx) == start:
                yield (ny, nx, t.depth+1), 1
            elif ny < 1 or ny > shape[0] - 2:
                continue
            elif nx < 1 or nx > shape[1] - 2:
                continue
            elif (ny, nx) in blizzards:
                continue

            yield (ny, nx, t.depth+1), 1

    traversal = nog.TraversalShortestPaths(next_edges)
    for vertex in traversal.start_from((start[0], start[1], 0), build_paths=True):
        if vertex[0:2] == end:
            #print(len(traversal.paths[vertex]) #, traversal.paths[vertex])
            break
    #print(traversal.distance)
    #print(traversal.paths[found])

    return traversal.distance, map_cache[traversal.distance]

def part2(puzzle_input):
    total = 0

    map, blizzards, start, end = read_input(puzzle_input)
    minutes, blizzards = find_path(map.shape, blizzards, start, end)
    total += minutes
    print("Trip There")
    print(minutes)
    print_blizzards(map.shape, blizzards)

    minutes, blizzards = find_path(map.shape, blizzards, end, start)
    total += minutes
    print("Trip Back")
    print(minutes)
    print_blizzards(map.shape, blizzards)

    minutes, blizzards = find_path(map.shape, blizzards, start, end)
    total += minutes
    print("Trip There Again")
    print(minutes)
    print_blizzards(map.shape, blizzards)
    
    return total

#ans = part2(os.path.join(os.path.join("..", "dat", "day24_test.txt")))
#assert ans == 54

ans = part2(os.path.join(os.path.join("..", "dat", "day24.txt")))
print(ans)

(0, 1) (36, 100)
Generating map at depth 3
Generating map at depth 4
Generating map at depth 5
Generating map at depth 6
Generating map at depth 7
Generating map at depth 8
Generating map at depth 9
Generating map at depth 10
Generating map at depth 11
Generating map at depth 12
Generating map at depth 13
Generating map at depth 14
Generating map at depth 15
Generating map at depth 16
Generating map at depth 17
Generating map at depth 18
Generating map at depth 19
Generating map at depth 20
Generating map at depth 21
Generating map at depth 22
Generating map at depth 23
Generating map at depth 24
Generating map at depth 25
Generating map at depth 26
Generating map at depth 27
Generating map at depth 28
Generating map at depth 29
Generating map at depth 30
Generating map at depth 31
Generating map at depth 32
Generating map at depth 33
Generating map at depth 34
Generating map at depth 35
Generating map at depth 36
Generating map at depth 37
Generating map at depth 38
Generating map at 